In [1]:
import numpy as np
import scipy.linalg as sla
from copy import deepcopy

In [160]:
dataset = {
    'a': np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
    'b': np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
    'c': np.array([
        [  5.,  3.,  40., -2.,  -3.,  -6., -2.,  2.,  3., 11., -19., -1.],
        [  4.,  3.,   4.,  6.,  -3.,   6., -7.,  8., -9., 10.,  -7.,  2.],
        [ -5.,  9.,  20., -2.,  -3.,   6.,  7.,  8.,  0.,  0.,   1., -2.],
        [ -2.,  3.,   0.,  4.,  -3.,   0., -1.,  4., -9.,  2.,   1.,  2.],
        [ -4., -2.,  14.,  2.,   3.,   6.,  0.,  1., -4., 10.,  -1.,  2.],
        [  2., -3.,   4.,  0.,  -3.,   4.,  7.,  8., -9.,  5.,   1., -8.],
        [  1.,  3.,   0.,  2.,   3.,   2.,  4.,  1.,  0., -1.,  -3., -2.],
        [ -5., -3.,   3., -2.,  -3.,  16.,  7.,  2., -9.,  1.,   1.,  2.],
        [ -1.,  3.,   4., -1.,  -3.,   6.,  0.,  0., -9.,  6.,   1.,  6.],
        [-12., -2.,   0., -2.,  -3.,   6.,  7.,  8.,  9.,  1.,  10.,  2.],
        [  3.,  2.,   1., -9., -13., -16.,  4.,  1.,  3., 11.,  15.,  2.],
        [ -3.,  5.,  12.,  4., -14., -15., 21., 11., -1., 22.,  12.,  2.],
    ]),
}
eps = 1e-9

In [131]:
def north_west(dataset):
    a = deepcopy(dataset['a'])
    b = deepcopy(dataset['b'])
    c = deepcopy(dataset['c'])
    i = 0
    j = 0
    plan = {}
    x = np.zeros(c.shape)
    for _ in range(a.shape[0] + b.shape[0] - 1):
        if i + 1 == a.shape[0]:
            x[i, j] = b[j]
        elif j + 1 == b.shape[0]:
            x[i, j] = a[i]
        else:
            x[i, j] = min(a[i], b[j])
        plan[(i, j)] = x[i, j]
        if x[i, j] == a[i]:
            b[j] -= x[i, j]
            i += 1
        else:
            a[i] -= x[i, j]
            j += 1
    return plan

In [129]:
def min_element(dataset):
    a = deepcopy(dataset['a'])
    b = deepcopy(dataset['b'])
    c = np.array(deepcopy(dataset['c']))
    plan = {}
    for _ in range(a.shape[0] + b.shape[0] - 1):
        min_value = np.argmin(c)
        j, i = min_value % c.shape[0], min_value // c.shape[1]
        x = min(a[i], b[j])
        if x == a[i]:
            c[i, :] = np.inf
            b[j] -= x
        else:
            c[:, j] = np.inf
            a[i] -= x
        
        plan[(i, j)] = x
    
    return plan

In [133]:
min_element(dataset)

{(0, 10): 1.0,
 (10, 5): 1.0,
 (11, 5): 0.0,
 (11, 4): 1.0,
 (9, 0): 1.0,
 (1, 8): 1.0,
 (3, 8): 0.0,
 (5, 11): 1.0,
 (2, 0): 0.0,
 (2, 4): 0.0,
 (6, 10): 0.0,
 (7, 1): 1.0,
 (2, 3): 1.0,
 (4, 1): 0.0,
 (6, 11): 0.0,
 (3, 6): 1.0,
 (6, 9): 1.0,
 (8, 3): 0.0,
 (4, 6): 0.0,
 (8, 7): 1.0,
 (4, 7): 0.0,
 (4, 9): 0.0,
 (4, 2): 1.0}

In [ ]:
def get_cycle(c, init_plan, i0, j0):
    plan = deepcopy(init_plan)
    c = np.zeros(c.shape)
    for i, j in init_plan.keys():
        c[i, j] = 1.
    
    c[i0, j0] = 1.
    

In [159]:
def solve(dataset, init_plan, max_iters=1, verbose=True):
    a = deepcopy(dataset['a'])
    b = deepcopy(dataset['b'])
    c = deepcopy(dataset['c'])
    
    for it in range(max_iters):
        
        # Step 1
        u, v = np.array([np.inf] * a.shape[0]), np.array([np.inf] * b.shape[0])
        u[0] = 0.
        while np.any(u == np.inf) or np.any(v == np.inf):
            for i in range(a.shape[0]):
                for j in range(b.shape[0]):
                    if (i, j) in init_plan:
                        if u[i] != np.inf and v[j] == np.inf:
                            v[j] = c[i, j] - u[i]
                        elif u[i] == np.inf and v[j] != np.inf:
                            u[i] = c[i, j] - v[j]
        
        # Step 2
        delta_j = -c + u.reshape(1, -1)[0, :] + v.reshape(-1, 1)[:, 0][:, np.newaxis]
        for i, j in init_plan.keys():
            delta_j[i, j] = -np.inf
        
        # Step 3
        if np.all(delta_j < -eps):
            return init_plan
        
        # Step 4
        i0, j0 = np.argmax(delta_j) // a.shape[0], np.argmax(delta_j) % b.shape[0]
        

In [151]:
solve(dataset, min_element(dataset))

TypeError: 'axis' is an invalid keyword to ufunc 'add'